## Decoding Arrest Decisions in Terry Traffic Stops

## Project Overview

The Terry Traffic Stops project aims to unravel the complex dynamics behind police decisions during traffic stops, particularly focusing on whether an arrest is made. Building on the legal precedent set by Terry v. Ohio, which introduced the concept of "reasonable suspicion," the project develops a machine learning model to predict the likelihood of arrest following a stop.

Using a rich dataset that includes details such as the stop’s context, demographics, and circumstances, the model seeks to identify patterns and key factors influencing these critical decisions. Beyond understanding police behavior, the project aspires to inform policy, promote fairness, and enhance transparency in law enforcement practices. Through this analysis, the Terry Traffic Stops project contributes to the broader conversation on policing and civil rights.

## Business Problem

Terry Stops, rooted in the principle of “reasonable suspicion,” allow police officers to temporarily detain individuals for investigation. However, the decision to escalate a stop to an arrest can be influenced by a variety of factors, some of which may not be immediately clear or consistent. This project seeks to address the complexities involved in these critical decisions through a multifaceted approach:

1. **Identifying Key Factors**: The project aims to determine the primary factors that influence whether an arrest is made following a Terry Stop. These factors could range from situational elements, such as time and location, to individual characteristics, including behavior and demeanor during the stop. Understanding these variables is crucial for identifying patterns and ensuring that decisions are made based on objective criteria.

2. **Predictive Modeling**: To systematically analyze these factors, the project will develop a binary classification model capable of predicting the likelihood of an arrest. By leveraging historical data, this model will provide a data-driven approach to anticipate outcomes, allowing law enforcement agencies to understand the potential impact of various factors on arrest decisions.

3. **Policy Evaluation**: A key focus of the project is to provide insights into the role of demographic factors, such as race and gender, in arrest decisions. By examining these elements, the project will help ensure that arrest decisions are fair and unbiased, fostering greater transparency and addressing ethical concerns in policing practices.

4. **Resource Allocation**: Understanding the patterns and factors that lead to arrests can also assist law enforcement agencies in better allocating their resources. By identifying areas where arrests are more likely or understanding the circumstances that typically lead to such outcomes, agencies can deploy their personnel more effectively and make informed decisions about training and policy development.


## Research Questions:

1. **Prediction Accuracy**: How well can we predict whether a traffic stop will occur using factors like location, time of day, and demographic information?

2. **Key Influences**: Which factors are most important in predicting traffic stops? Are there any surprising trends or patterns?

3. **Bias and Fairness**: Does the model show any bias in its predictions? For example, does it unfairly target certain groups of drivers more than others?


## Data Understanding Overview
In this phase, we explore the Terry Traffic Stops dataset to gain insights into the variables that describe police interactions during traffic stops. The dataset includes detailed information on both the individuals stopped and the officers involved, covering demographics, stop outcomes, and geographical data. By examining these features, we aim to identify key factors that influence stop outcomes, assess potential biases, and prepare the data for further analysis and modeling

1. **Subject Age Group**: Categorizes the age of the individual involved in the stop.
2. **Subject ID**: Unique identifier for each individual stopped.
3. **GO / SC Num**: Case or report number associated with the stop.
4. **Terry Stop ID**: Unique identifier for each specific Terry Stop event.
5. **Stop Resolution**: Outcome of the stop (e.g., arrest, warning).
6. **Weapon Type**: Type of weapon found, if any, during the stop.
7. **Officer ID**: Unique identifier for the officer conducting the stop.
8. **Officer YOB**: Year of birth of the officer involved in the stop.
9. **Officer Gender**: Gender of the officer involved in the stop.
10. **Officer Race**: Race of the officer conducting the stop.
11. **Subject Perceived Race**: Race of the individual as perceived by the officer.
12. **Subject Perceived Gender**: Gender of the individual as perceived by the officer.
13. **Reported Date**: Date when the stop was reported.
14. **Reported Time**: Time of day when the stop was reported.
15. **Initial Call Type**: Reason for the initial call that led to the stop.
16. **Final Call Type**: Nature of the call after the stop was resolved.
17. **Call Type**: General category of the call or incident.
18. **Officer Squad**: The squad or unit to which the officer belongs.
19. **Arrest Flag**: Indicator of whether the stop resulted in an arrest.
20. **Frisk Flag**: Indicator of whether the individual was frisked during the stop.
21. **Precinct**: The police precinct where the stop took place.
22. **Sector**: Sub-division within a precinct for geographical analysis.
23. **Beat**: Smallest geographical area of police patrol.


## Step 1: Import all the Necessary Libraries

In [20]:
# Importing essential libraries for data handling and analysis
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [21]:
# Load the dataset and print out the first few rows of our data
df = pd.read_csv("Terry_Traffic_Stops.csv")

# Display the first few rows to verify the data
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,36 - 45,7732696346,20190000315233,9803669705,Field Contact,-,4161,1957,M,American Indian/Alaska Native,...,10:47:39.0000000,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 1ST W - KING - PLATOON 1,N,N,West,K,K2
1,46 - 55,8295859194,20190000207006,8299762394,Arrest,-,6404,1971,M,White,...,17:08:12.0000000,BURG - OCCUPIED RESD,--PROPERTY DEST (DAMG),911,NORTH PCT 2ND W - NORA (JOHN) - PLATOON 1,Y,N,North,N,N3
2,26 - 35,-1,20170000002886,467843,Field Contact,NaN,7430,1984,F,White,...,17:48:00.0000000,-,-,-,NORTH PCT 2ND WATCH - NORTH BEATS,N,Y,-,-,-
3,1 - 17,-1,20180000275743,472723,Offense Report,NaN,5151,1962,M,White,...,11:49:00.0000000,-,-,-,SOUTHWEST PCT 1ST W - WILLIAM - PLATOON 2,N,N,Southwest,F,F1
4,36 - 45,16227498273,20220000263279,37099192062,Arrest,-,7655,1982,M,Nat Hawaiian/Oth Pac Islander,...,22:18:36.0000000,SHOPLIFT - THEFT,--BURGLARY - NON RESIDENTIAL/COMMERCIAL,ONVIEW,SOUTHWEST PCT 2ND W - FRANK - PLATOON 2,Y,N,Southwest,F,F2


Lets have a general overview of our dataset

In [22]:

class DataFrameInspector:
    def __init__(self, file_path):
        # Initialize the class with the file path and load the DataFrame
        self.file_path = file_path
        self.df = pd.read_csv(file_path)

    def display_info(self):
        # Display basic information about the DataFrame
        print("DataFrame Info:")
        self.df.info()

    def show_head(self, n=5):
        # Show the first n rows of the DataFrame
        print(f"\nFirst {n} rows:")
        print(self.df.head(n))

    def show_basic_stats(self):
        # Display summary statistics for numerical columns
        print("\nBasic statistics:")
        print(self.df.describe())

    def count_null_values(self):
        # Count the number of null values in each column
        print("\nNull values per column:")
        print(self.df.isnull().sum())

    def count_unique_values(self):
        # Count the number of unique values in each column
        print("\nUnique values per column:")
        print(self.df.nunique())

    def inspect_all(self):
        # Run all inspection methods
        self.display_info()
        self.show_head()
        self.show_basic_stats()
        self.count_null_values()
        self.count_unique_values()

inspector = DataFrameInspector('Terry_Traffic_Stops.csv')
inspector.inspect_all()

# Alternatively, you can call individual methods:
# inspector.display_info()
# inspector.show_head(10)  # Show first 10 rows
# inspector.show_basic_stats()
# inspector.count_null_values()
# inspector.count_unique_values()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61021 entries, 0 to 61020
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         61021 non-null  object
 1   Subject ID                61021 non-null  int64 
 2   GO / SC Num               61021 non-null  int64 
 3   Terry Stop ID             61021 non-null  int64 
 4   Stop Resolution           61021 non-null  object
 5   Weapon Type               28456 non-null  object
 6   Officer ID                61021 non-null  object
 7   Officer YOB               61021 non-null  int64 
 8   Officer Gender            61021 non-null  object
 9   Officer Race              61021 non-null  object
 10  Subject Perceived Race    61021 non-null  object
 11  Subject Perceived Gender  61021 non-null  object
 12  Reported Date             61021 non-null  object
 13  Reported Time             61021 non-null  object
 14  Initia

## Data Understanding: General overview

1. Dataset Overview

The Terry Traffic Stops dataset contains a comprehensive record of 61,021 police stops. This data is organized across 23 columns, providing a detailed view of each incident.

2. Data Structure

The dataset's structure comprises 4 integer columns and 19 object columns. The object columns likely contain strings or mixed data types, allowing for a diverse range of information to be captured.

3. Missing Data

While most columns are complete, there are two notable exceptions. The "Weapon Type" column has the highest number of null values at 32,565, suggesting that weapon information is not always available or applicable. Additionally, the "Officer Squad" column has 561 missing entries.

4. Key Column Categories

4.1 Subject Information
The dataset includes details about the subjects of the stops, such as their age group, ID, perceived race, and perceived gender.

4.2 Officer Details
Information about the officers conducting the stops is also recorded, including their ID, year of birth (YOB), gender, and race.

4.3 Stop Characteristics
Each stop is documented with a unique Terry Stop ID, along with the stop resolution and the reported date and time.

4.4 Location Data
The geographic context of each stop is captured through precinct, sector, and beat information.

5. Numerical Insights

The "Officer YOB" column provides interesting demographic data about the police force. Birth years range from 1900 to 2002, with a mean of 1984. Both the Subject ID and Terry Stop ID columns show a wide range of values, likely serving as unique identifiers for each entry.

6. Categorical Data Highlights

Several columns offer insights into the nature of the stops and the diversity of those involved:
- Stop Resolution has 5 unique categories, indicating various outcomes of the stops.
- There are 9 distinct precincts represented in the data.
- Officer Race is categorized into 9 groups.
- Subject Perceived Race has 11 unique categories, suggesting a detailed approach to recording racial data.

7. Flag Columns

The dataset includes two flag columns that provide quick reference points:
- Arrest Flag is binary, indicating whether an arrest was made.
- Frisk Flag has 3 unique values, potentially indicating different levels or types of frisking procedures.

## Column classification

In [23]:
def classify_columns(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Initialize lists to store column names
    numerical_columns = []
    categorical_columns = []

    # Iterate through each column
    for column in df.columns:
        # Check if the column is numerical
        if df[column].dtype in ['int64', 'float64']:
            numerical_columns.append(column)
        else:
            # Check if the column is categorical
            if df[column].dtype == 'object' or df[column].dtype.name == 'category':
                categorical_columns.append(column)
            # If it's neither numerical nor categorical (e.g., datetime), you can add more conditions here

    # Print the results
    print("Numerical columns:")
    for col in numerical_columns:
        print(f"- {col}")

    print("\nCategorical columns:")
    for col in categorical_columns:
        print(f"- {col}")

    # Return the lists if you need to use them later
    return numerical_columns, categorical_columns

# Usage
file_path = 'Terry_Traffic_Stops.csv'
num_cols, cat_cols = classify_columns(file_path)

Numerical columns:
- Subject ID
- GO / SC Num
- Terry Stop ID
- Officer YOB

Categorical columns:
- Subject Age Group
- Stop Resolution
- Weapon Type
- Officer ID
- Officer Gender
- Officer Race
- Subject Perceived Race
- Subject Perceived Gender
- Reported Date
- Reported Time
- Initial Call Type
- Final Call Type
- Call Type
- Officer Squad
- Arrest Flag
- Frisk Flag
- Precinct
- Sector
- Beat


### Dataset Classification Overview

The dataset is composed of two primary types of columns: **Numerical** and **Categorical**.

- **Numerical Columns:** These columns consist of data that represent measurable quantities or unique identifiers. They typically include values that can be used for mathematical operations, such as calculating averages or identifying unique instances.

- **Categorical Columns:** These columns contain data that represent distinct groups or categories. Categorical data is used to classify and label information, often representing qualitative aspects of the dataset, such as characteristics, types, or outcomes.

Understanding these categories is crucial for selecting the right analytical and modeling techniques, as numerical and categorical data often require different approaches.


# Step 2: Data Cleaning

Identify and handle missing values

Scrub the data to remove any inaccuracies or inconsistencies

Transform and organize the data into a standardized format for analysis

Enhance the quality and usability of the data through restructuring and formatting adjustments.

## 2.1 Handling Missing Values

In [24]:
#Checking correlation between missing values and another feature
df['Weapon_Type_Missing'] = df['Weapon Type'].isna()
df.groupby('Precinct')['Weapon_Type_Missing'].mean()


Precinct
-            0.813030
East         0.505514
FK ERROR     0.000000
North        0.579929
OOJ          0.000000
South        0.497290
Southwest    0.460560
Unknown      0.950000
West         0.371315
Name: Weapon_Type_Missing, dtype: float64

### Interpretation of Missing Values in the "Weapon Type" Column

The output shows the mean proportion of missing values in the **Weapon Type** column for each **Precinct**. Here’s what the results imply:

1. **High Proportion of Missing Values in Some Precincts:**
   - **"-" Precinct:** About **81.3%** of the rows for this precinct have missing values in the **Weapon Type** column. This is a very high proportion, indicating that most data for this precinct is missing for this feature.
   - **"Unknown" Precinct:** About **95%** of the data for this precinct is missing in the **Weapon Type** column, suggesting a severe lack of data for this feature in this group.

2. **Moderate Proportion of Missing Values:**
   - **East, North, South Precincts:** Around **50-58%** of the data is missing in the **Weapon Type** column for these precincts, which is a significant amount that might affect the model if these rows are dropped.

3. **Low to No Missing Values:**
   - **"FK ERROR" and "OOJ" Precincts:** Both have **0%** missing values, indicating that all data for the **Weapon Type** column is available.
   - **"West" and "Southwest" Precincts:** Have around **37.1% and 46.1%** missing values, respectively, which is still substantial but lower than in other precincts.

### Implications for the Data and the Model

- **Non-Random Missing Data:**  
  The varying proportions of missing values across precincts suggest that the missingness is not random. Certain precincts, such as **"-"** and **"Unknown"**, have a much higher likelihood of missing **Weapon Type** data, which could indicate underlying patterns or data collection issues. Dropping these rows could introduce bias into the model, particularly against precincts with high rates of missing data, potentially leading to skewed predictions.

- **Potential Bias Introduction:**  
  Since missing values are not uniformly distributed, removing them could disproportionately affect the representation of certain precincts. This could reduce the model’s accuracy or fairness by excluding relevant data, especially if **Weapon Type** is an important predictor of the outcome.

### Actionable Next Steps

- **Imputation Strategy:**  
  Consider imputing missing values using a placeholder ("Unknown") or filling in based on patterns in the data (e.g., the most common **Weapon Type** within each precinct).

- **Feature Engineering:**  
  Create a binary feature indicating whether **Weapon Type** is missing, as this could provide valuable information about the nature of stops in specific precincts.


In [25]:
# Feature engineering
# 1. Create a binary indicator for missing 'Weapon Type'
df['Weapon_Type_Missing'] = df['Weapon Type'].isna().astype(int)

# 2. Impute missing 'Weapon Type' values with a placeholder
df['Weapon Type'].fillna('Unknown', inplace=True)

# 3. Feature engineering: Create interaction feature between 'Weapon Type' and 'Precinct'
df['Weapon_Precinct_Interaction'] = df['Weapon Type'] + '_' + df['Precinct']

# 4. Feature engineering: Extract date-related features from 'Reported Date'
df['Reported_Year'] = pd.to_datetime(df['Reported Date']).dt.year
df['Reported_Month'] = pd.to_datetime(df['Reported Date']).dt.month
df['Reported_Day'] = pd.to_datetime(df['Reported Date']).dt.day

# 5. Feature engineering: Extract time-related features from 'Reported Time'
# Use `errors='coerce'` to handle any inconsistent formats
df['Reported_Hour'] = pd.to_datetime(df['Reported Time'], errors='coerce').dt.hour

# Verify the newly created features
print(df[['Weapon Type', 'Weapon_Type_Missing', 'Weapon_Precinct_Interaction', 
          'Reported_Year', 'Reported_Month', 'Reported_Day', 'Reported_Hour']].head())


C:\Users\ronoe\AppData\Local\Temp\ipykernel_31764\2960947321.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Weapon Type'].fillna('Unknown', inplace=True)
C:\Users\ronoe\AppData\Local\Temp\ipykernel_31764\2960947321.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Reported_Hour'] = pd.to_datetime(df['Reported Time'], errors='coerce').dt.hour


  Weapon Type  Weapon_Type_Missing Weapon_Precinct_Interaction  Reported_Year  \
0           -                    0                      -_West           2019   
1           -                    0                     -_North           2019   
2     Unknown                    1                   Unknown_-           2017   
3     Unknown                    1           Unknown_Southwest           2018   
4           -                    0                 -_Southwest           2022   

   Reported_Month  Reported_Day  Reported_Hour  
0               8            24             10  
1               6             7             17  
2               7             7             17  
3               8            20             11  
4              10             1             22  


In [26]:
# Count missing values before dropping
missing_count = df['Officer Squad'].isnull().sum()

# Drop rows with missing values in 'Officer Squad'
df = df.dropna(subset=['Officer Squad'])

# Verify the drop
remaining_count = len(df)

In our analysis of the Terry Traffic Stops dataset, we encountered missing values in the 'Officer Squad' column. After careful consideration, we decided to drop these rows with missing values. Here's why:

### 1. Small Proportion of Missing Data

- Total rows in the dataset: 61,021
- Rows with missing 'Officer Squad' data: 561
- Percentage of missing data: 0.92%

With less than 1% of the data missing, removing these rows is unlikely to significantly impact our analysis or introduce bias.

### 2. Preserving Data Integrity

By removing incomplete records, we ensure that all remaining data points have complete information for the 'Officer Squad' variable. This is crucial for maintaining the integrity of our analysis, especially if the officer's squad is an important factor in our study.

### 3. Avoiding Assumptions

Dropping the rows avoids making potentially incorrect assumptions about the missing data. Methods like imputation or creating a separate 'Unknown' category might introduce bias or noise into our analysis.

### 4. Simplicity and Transparency

This approach is straightforward and easy to explain, enhancing the transparency and reproducibility of our analysis. It doesn't require complex imputation techniques that might be difficult to justify or interpret.

### 5. Minimal Impact on Statistical Power

Given the large size of our dataset (over 60,000 rows), removing 561 rows is unlikely to significantly reduce the statistical power of our analyses.

### 6. Clean Dataset for Analysis

By working with a complete dataset, we simplify subsequent analysis steps and reduce the risk of errors or unexpected behavior due to missing values.

### Impact on Analysis

- Final dataset size: 60,460 rows
- All subsequent analyses will be performed on this cleaned dataset


## 2.2 Dropping irrelevant columns

We'll drop columns that don't affect customer churn (RowNumber, CustomerId, Surname).

In [31]:
# Drop irrelevant columns
irrelevant_columns = ['Subject ID', 'GO / SC Num', 'Terry Stop ID', 'Officer ID', 'Officer YOB', 'Officer Squad', 'Beat']
df = df.drop(irrelevant_columns, axis=1)

In [35]:
# confirm that the columns have been dropped
df.head()

,Subject Age Group,Stop Resolution,Weapon Type,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Reported Date,Reported Time,Initial Call Type,...,Arrest Flag,Frisk Flag,Precinct,Sector,Weapon_Type_Missing,Weapon_Precinct_Interaction,Reported_Year,Reported_Month,Reported_Day,Reported_Hour
0,36 - 45,Field Contact,-,M,American Indian/Alaska Native,-,Male,2019-08-24T00:00:00Z,10:47:39.0000000,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,...,N,N,West,K,0,-_West,2019,8,24,10
1,46 - 55,Arrest,-,M,White,White,Male,2019-06-07T00:00:00Z,17:08:12.0000000,BURG - OCCUPIED RESD,...,Y,N,North,N,0,-_North,2019,6,7,17
2,26 - 35,Field Contact,Unknown,F,White,White,Male,2017-07-07T00:00:00Z,17:48:00.0000000,-,...,N,Y,-,-,1,Unknown_-,2017,7,7,17
3,1 - 17,Offense Report,Unknown,M,White,Black or African American,Female,2018-08-20T00:00:00Z,11:49:00.0000000,-,...,N,N,Southwest,F,1,Unknown_Southwest,2018,8,20,11
4,36 - 45,Arrest,-,M,Nat Hawaiian/Oth Pac Islander,White,Male,2022-10-01T00:00:00Z,22:18:36.0000000,SHOPLIFT - THEFT,...,Y,N,Southwest,F,0,-_Southwest,2022,10,1,22
